In [1]:
import pandas as pd

In [2]:
def preprocess_clan(path):
    clan = pd.read_csv(path)
    clan.drop(clan.columns[0], axis = 'columns', inplace= True) #drop an extra index column
    return clan

In [3]:
brujah = preprocess_clan("/home/anton/Documents/projects/vtm_exploration/vampire/data/Brujah.csv")

In [4]:
def generate_edges(clan):
    edges = []
    for vamp in clan.iterrows():
        #print(vamp[1]["Sire"])

        if vamp[1]["Sire"] in list(clan["Name"]):
            edges.append((vamp[1]["Sire"], vamp[1]["Name"]))
    
    return edges

In [5]:
brujah_edges = generate_edges(clan=brujah)

In [6]:
import re
import numpy as np

def select_generation(gen_str):

    gen_str = str(gen_str)

    gen_str = gen_str.replace("V5", "") #deal with the edge cases where V5 system is mentioned
    gen_str = re.sub(r'\[\d+\]', "", gen_str) #deal with source reference (of the type [4])
    gen = re.findall(r'\d+', gen_str)

    gen = [int(x) for x in gen if int(x) <= 16 or int(x) > 0]

    if not gen: 
        return np.nan

    if "diab" in gen_str: #check for diablerie 
        gen = min(gen)
    else: 
        gen = max(gen)

    return int(gen)

brujah["Generation"] = brujah["Generation"].apply(lambda x: select_generation(str(x)))

### Generate Graphs

In [11]:
red_gradient = {
    1: "#8b0000",    # Darkest Red
    2: "#920c0c",
    3: "#991919",
    4: "#a02626",
    5: "#a83333",
    6: "#af3f3f",
    7: "#b64c4c",
    8: "#bd5959",
    9: "#c46666",
    10: "#cc7272",
    11: "#d37f7f",
    12: "#da8c8c",
    13: "#e29999",
    14: "#e9a5a5",
    15: "#f0b2b2",
    16: "#f7bfbf",
    17: "#ffcccc"     # Lightest Red
}

background_color = "#222222"

In [ ]:
from pyvis.network import Network
from pyvis.options import Layout
import networkx as nx
from matplotlib import pyplot as plt

background_color = "#222222"

net = Network(height="800px", directed=True, bgcolor=background_color, layout=True, filter_menu=True, cdn_resources='remote')

brujah_temp = brujah.sort_values(by="Generation")
brujah_temp


for index, vamp in brujah_temp.iterrows():

    if not np.isnan(vamp["Generation"]): 
        level = vamp["Generation"]
        net.add_node(n_id=vamp["Name"],
                    label=vamp["Name"] + "\n" + str(int(level)) + " gen",
                    level = level, 
                    color=red_gradient[level],
                    shape = "box",
                    title = vamp["Link"], #title=f"<a href='{vamp["Link"]}' target='_blank'>{vamp["Name"]} \n {int(level)} gen</a>",
                    font = {"color":"white"})


for index, vamp in brujah_temp.iterrows():
    if np.isnan(vamp["Generation"]):
        level = np.random.binomial(n = 15, p = 0.5)
        net.add_node(n_id=vamp["Name"],
                     level = level, 
                     color="#BEBEBE")

for edge in brujah_edges:
    net.add_edge(edge[0], edge[1])

net.set_options("""options= {
        "layout": {
            "hierarchical": {
            "enabled": true,
            "nodeSpacing": 15,
            "treeSpacing": 10,
            "direction": "LR"
            }
        }
    }
        """)

#net.toggle_physics(False)
    
net.show("graphs/brujah.html")

AttributeError: 'dict' object has no attribute 'physics'

In [81]:
print(net.nodes)

[{'color': '#991919', 'level': 3.0, 'title': 'https://whitewolf.fandom.com/wiki/Brujah_Antediluvian', 'font': {'color': 'white'}, 'id': 'Brujah Antediluvian', 'label': 'Brujah Antediluvian\n3 gen', 'shape': 'box'}, {'color': '#991919', 'level': 3.0, 'title': 'https://whitewolf.fandom.com/wiki/Troile', 'font': {'color': 'white'}, 'id': 'Troile', 'label': 'Troile\n3 gen', 'shape': 'box'}, {'color': '#a02626', 'level': 4.0, 'title': 'https://whitewolf.fandom.com/wiki/Gersakkun', 'font': {'color': 'white'}, 'id': 'Gersakkun', 'label': 'Gersakkun\n4 gen', 'shape': 'box'}, {'color': '#a02626', 'level': 4.0, 'title': 'https://whitewolf.fandom.com/wiki/Azif_(Brujah)', 'font': {'color': 'white'}, 'id': 'Azif (Brujah)', 'label': 'Azif (Brujah)\n4 gen', 'shape': 'box'}, {'color': '#a02626', 'level': 4.0, 'title': 'https://whitewolf.fandom.com/wiki/Hannibal_(Brujah)', 'font': {'color': 'white'}, 'id': 'Hannibal (Brujah)', 'label': 'Hannibal (Brujah)\n4 gen', 'shape': 'box'}, {'color': '#a02626', '

In [14]:
def inject_js(html_path, original_pattern, new_pattern):    
    with open(html_path, 'r', encoding='utf-8') as file: 
        source_code = file.read()
        source_code_updated = source_code.replace(original_pattern, new_pattern)
    
    with open(html_path, 'w', encoding='utf-8') as f:
        f.write(source_code_updated)

In [ ]:
""" 
let link = node.title
                                // Send POST request
                                fetch(window.location.href, {
                                method: 'POST',
                                headers: {'Content-Type': 'application/json'},
                                body: JSON.stringify({link: link})
                                })
                                .then(response => {
                                if (!response.ok) throw new Error('Network response was not ok');
                                return response.text();
                                })
                                .then(data => {
                                console.log('POST succeeded:', data);
                                })
                                .catch(error => {
                                console.error('POST failed:', error);
                                });
  """

In [37]:
import networkx as nx
from pyvis.network import Network

G = nx.Graph()

for index, vamp in brujah.iterrows():

    if not np.isnan(vamp["Generation"]): 
        level = vamp["Generation"]
        G.add_node(node_for_adding=vamp["Name"],
                    label=vamp["Name"] + "\n" + str(int(level)) + " gen",
                    level = level, 
                    color=red_gradient[level],
                    shape = "box",
                    title = vamp["Link"], #title=f"<a href='{vamp["Link"]}' target='_blank'>{vamp["Name"]} \n {int(level)} gen</a>",
                    font = {"color":"white"},
                    Generation = level,
                    Name = vamp["Name"])


for index, vamp in brujah.iterrows():
    if np.isnan(vamp["Generation"]):
        level = np.random.binomial(n = 15, p = 0.5)
        G.add_node(node_for_adding=vamp["Name"],
                     level = level, 
                     color="#FFFFFF",
                     font = {"color":"black"},
                     title = vamp["Link"],
                     Name = vamp["Name"],
                     shape = "box")
        
#for edge in brujah_edges:
#    G.add_edge(*edge)#, Childer_Of=edge[0], Sire_Of=edge[1])

#pos = nx.planar_layout(G)
pos = nx.arf_layout(G)


#for node, (x, y) in pos.items():
   # G.nodes[node]['x'] = float(x)   
   # G.nodes[node]['y'] = float(y)

print(G.edges)


net_temp = Network(height="800px", directed=True, bgcolor=background_color,  filter_menu=True, cdn_resources='remote')
net_temp.from_nx(G)

for edge in brujah_edges:
    net_temp.add_edge(edge[0], edge[1], Childer_Of=edge[0], Sire_Of=edge[1])



#print(net_temp.edges)

net_temp.save_graph("graphs/brujah.html")

[]


Error: Failed to open Wayland display, fallback to X11. WAYLAND_DISPLAY='wayland-0' DISPLAY=':0'


In [ ]:
 # set the background of the page to match the graph
inject_js("graphs/brujah.html", 
          original_pattern = "<html>\n ", 
          new_pattern = f'<html>\n <body style="background-color:{background_color};">\n') 

# update the border to blend in with the background
inject_js("graphs/brujah.html", 
          original_pattern='<style type="text/css">', 
          new_pattern=f'<style type="text/css">\n\t\t\t#mycard {{\n \t\t\t\tbackground-color: {background_color};\n\t\t\t\n  border: 1px solid {background_color};\n}}\n #loadingBar {{ display: none !important;}}\n')

inject_js("graphs/brujah.html", 
          original_pattern='<div class="card" style="width: 100%">', 
          new_pattern='<div id="mycard" class="card" style="width: 100%">')

inject_js("graphs/brujah.html", 
          original_pattern='<div class="card" style="width: 100%">', 
          new_pattern='<div id="mycard" class="card" style="width: 100%">')

inject_js("graphs/brujah.html", 
          original_pattern="border: 1px solid lightgray;", 
          new_pattern=f"border: 1px solid {background_color};")

# add double-click to the nodes, open a new page 

inject_js("graphs/brujah.html",
          original_pattern="network = new vis.Network(container, data, options);",
          new_pattern="""network = new vis.Network(container, data, options);
                        \n network.on("doubleClick", function(params) {
                        if (params.nodes.length > 0) {
                            var nodeId = params.nodes[0];
                            var node = nodes.get(nodeId);
                            console.log(node)
                            if (node && node.title) {
                                //window.open(node.title, "_blank");

                                //console.log(window.parent.parent)

                                //var outboundData = Object.assign({
                                //isStreamlitMessage: true,
                                //type: "link_wrong",
                                //}, node.title)


                                window.parent.parent.postMessage({ type: 'LINK', payload: node.title }, "*")
                            }
                        }   
                    });\n""")

# move the starting position of the graph to the top node

#inject_js("graphs/brujah.html", 
#          original_pattern='network.once("stabilizationIterationsDone", function() {', 
#          new_pattern= """network.once("stabilizationIterationsDone", function() {\n
#                            network.moveTo({
#                                position: {x: 0, y: 0},
#                                scale: 0.8
#                            });\n""")

# restrict filter options to only certain attributes

inject_js("graphs/brujah.html",
          original_pattern="propControl.addOption({id: eachProp, title: eachProp})",
          new_pattern="if (['Generation', 'Name', 'Sire_Of', 'Childer_Of'].includes(eachProp)){\npropControl.addOption({id: eachProp, title: eachProp})\n}\n")


